### BW37 - 내장 타입을 여러 단계로 내포시키보다는 클래스를 합성하자

In [1]:
class SimpleGradebook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = []

    def report_grade(self, name, score):
        self._grades[name].append(score)

    def average_grade(self, name):
        grades = self._grades[name]
        return sum(grades) / len(grades)

- 파이썬 내장 딕셔너리 타입을 사용하면 객체의 생명 주기 동안 동적인 내부 상태를 잘 유지
<br/> **동적?** 어떤값이 들어 올지 미리 알수 없는 식별자들을 유지

In [2]:
book = SimpleGradebook()
book.add_student('아이작 뉴턴')
book.report_grade('아이작 뉴턴', 90)
book.report_grade('아이작 뉴턴', 95)
book.report_grade('아이작 뉴턴', 85)

print(book.average_grade('아이작 뉴턴'))

90.0


<hr/>

Ex. 클래스를 확장해서 전체가 아닌 과목별 성적을 리스트로 저장하고 싶다면

In [6]:
from collections import defaultdict

class BySubjectGradebook:
    def __init__(self):
        self._grades = {}  # 외부 dict

    def add_student(self, name):
        self._grades[name] = defaultdict(list)  # 내부 dict

    def report_grade(self, name, subject, grade):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append(grade)

    def average_grade(self, name):
        by_subject = self._grades[name]
        total, count = 0, 0
        for grades in by_subject.values():
            total += sum(grades)
            count += len(grades)
        return total / count

Q.사용하기 쉬운데 왜 과하게 확장하면 깨지기 쉬운 코드를 작성할 위험성이 있는걸까?
- 내부상태에 원소가 없기에 defaultdict를 사용 원소(과목이 없는 경우)를 처리

In [7]:
book = BySubjectGradebook()
book.add_student('알버트 아인슈타인')
book.report_grade('알버트 아인슈타인', '수학', 75)
book.report_grade('알버트 아인슈타인', '수학', 65)
book.report_grade('알버트 아인슈타인', '체육', 90)
book.report_grade('알버트 아인슈타인', '체육', 95)
print(book.average_grade('알버트 아인슈타인'))

81.25


- 아직은  쓸만하다. <br/> Q. 다단계 딕셔너리를 처리가 무슨말?

<hr/>

Ex. 이제 요구사항이 바뀐다. 기말고사가 다른 쪽지 시험보다 성적에 더큰 영향(가중치)
<br/> 리스트를 튜플의 리스트로 매핑하도록 변경하는 것

In [ ]:
class WeightedGradebook:
    def __init__(self):
        self._grades = {}

    def add_student(self, name):
        self._grades[name] = defaultdict(list)

    def report_grade(self, name, subject, score, weight):
        by_subject = self._grades[name]
        grade_list = by_subject[subject]
        grade_list.append((score, weight))

- report_grade는 성적 리스트가 튜플 인스턴스를 저장하게 했을 뿐 단순하게 변경 되었다.
<br/> average_grade 메서드는 이중루프가 쓰이면서 읽기 어려워 졌다.

In [15]:
    def average_grade(self, name):
        by_subject = self._grades[name]
        score_sum, score_count = 0, 0

        for subject, scores in by_subject.items():
            subject_avg, total_weight = 0, 0

            for score, weight in scores:
                subject_avg += score * weight
                total_weight += weight

            score_sum += subject_avg / total_weight
            score_count += 1

        return score_sum / score_count

In [9]:
book = WeightedGradebook()
book.add_student('알버트 아인슈타인')
book.report_grade('알버트 아인슈타인', '수학', 75, 0.05)
book.report_grade('알버트 아인슈타인', '수학', 65, 0.15)
book.report_grade('알버트 아인슈타인', '수학', 70, 0.80)
book.report_grade('알버트 아인슈타인', '체육', 100, 0.40)
book.report_grade('알버트 아인슈타인', '체육', 85, 0.60)
print(book.average_grade('알버트 아인슈타인'))

80.25


- 클래스도 쓰기 어려워짐, 위치로 인자를 지정하면 값이 어떤 뜻을 가지는지 이해하기 어렵다.
- 그래서 이렇게 복잡하면 내장 타입(딕셔,튜플,집합,리스트)이 아닌
<br/> 클래스 계층 구조를 사용하자.

- 처음에 가중치가 부여된 성적을 지원해야 한다는 첫 번째 요구 사항을 몰랐다. <br/> → 새로운 클래스를 만들어 내는 번거로움을 감수할 이유가 없었다.
- 파이썬 내장 타입은 사용하기 편하므로 계층을 추가 해가면서 사용하기 쉽다. <br/> → 내포 단계가 2단계 이상되면 사용하지 말아야 한다.

- 코드가 점점 복잡해지고 있음을 알게되었으면 즉시 해당기능을 클래스로 분리 해야한다.

<hr/>

#### 클래스를 활용해 리팩터링하기
> 리팩터링 ?
- 외부 동작을 바꾸지 않으면서 내부 구조를 개선하는 방법입니다.
<br/> 겉으로 보이는 동작의 변화 없이 소프트웨어 구조를 바꾸다.

Ex. 리스트 안에 점수를 저장하기 위해 튜플을 사용한 예제

In [17]:
grades = []
grades.append((95, 0.45))
grades.append((85, 0.55))
total = sum(score * weight for score, weight in grades)
total_weight = sum(weight for _, weight in grades)
average_grade = total / total_weight

- total_weight를 계산할 때 '_'를 사용해 각 점수 튜플의 첫 번째 원소를 무시
<br/> Ex. 선생님이 메모를 추가해야 하는 경우

In [18]:
grades = []
grades.append((95, 0.45, '참 잘했어요'))
grades.append((85, 0.55, '조금 만 더 열심히'))
total = sum(score * weight for score, weight, _ in grades)
total_weight = sum(weight for _, weight, _ in grades)
average_grade = total / total_weight

- 원소가 늘어나면 다른 인덱스를 무시하기 위해 '_' 를 더 많이 써야한다.
<br/> → 기존의 튜플에서 처리 해야할 튜플이 3개에서 그이상으로 늘어기 때문에 <br/> 결론적으로 딕셔너리를 여러 단계로 내포시키는 경우와 유사하다고 볼수있다.

<hr/>

- collection 내장 모듈에 있는 namedtuple 타입이 이런 경우에 딱 들어 맞는다.

> namedtuple ? 
- 튜플의 성질은 가졌지만 항목에 이름으로 접근이 가능한 튜플

→ 작은 불변 데이터 클래스를 쉽게 정의할 수 있다.

In [ ]:
from collections import namedtuple
Grade = namedtuple('Grade', ('score', 'weight'))

- 클래스의 인스턴스를 만들 때는 위치 기반 인자를 사용해도 되고 키워드 인자를 사용해도 된다.
<br/> Q. 왜 둘다 사용해도 되는건지?
- 필드에 접근할 때는 애트리뷰트 이름을 쓸 수 있다.
<br/> → 요구 사항이 바뀌는 경우에 namedtuple을 클래스로 변경하기가 쉽다.

#### namedtuple 한계
- namedtuple 클래스에는 디폴트 인자 값을 지정할 수 없다
<br/> → 프로퍼티가 4~5개 보다 더 많아지면 dataclasses 내장 모듈을 사용하는 편이 낫다.
- 사용하는 모든 부분을 제어할 수 있는 상황이 아니라면 명시적으로 새로운 클래스를 정의하는 편이 더 낫다.
<br/> → 인스턴스의 애트리뷰트 값을 숫자 인덱스를 사용해 접근할 수 있고 이터레이션도 가능하다.

Ex. 일련의 점수를 포함하는 단일 과목을 표현하는 클래스

In [19]:


class Subject:
    def __init__(self):
        self._grades = []

    def report_grade(self, score, weight):
        self._grades.append(Grade(score, weight))

    def average_grade(self):
        total, total_weight = 0, 0
        for grade in self._grades:
            total += grade.score * grade.weight
            total_weight += grade.weight
        return total / total_weight


class Student: # 다음으로 한 학생이 수강하는 과목들을 표현하는 클래스
    def __init__(self):
        self._subjects = defaultdict(Subject)

    def get_subject(self, name):
        return self._subjects[name]

    def average_grade(self):
        total, count = 0, 0
        for subject in self._subjects.values():
            total += subject.average_grade()
            count += 1
        return total / count # 모든 학생을 저장하는 컨테이너를 만들 수 있다.


class Gradebook:
    def __init__(self):
        self._students = defaultdict(Student) # 이름을 사용해서 동적으로 학생을 지정

    def get_student(self, name):
        return self._students[name]

In [13]:
book = Gradebook()
albert = book.get_student('알버트 아인슈타인')
math = albert.get_subject('수학')
math.report_grade(75, 0.05)
math.report_grade(65, 0.15)
math.report_grade(70, 0.80)
gym = albert.get_subject('체육')
gym.report_grade(100, 0.40)
gym.report_grade(85, 0.60)
print(albert.average_grade())

80.25


- 코드 줄수는 두 배 이상이지만, 코드를 읽기 쉽고, 확장성이 좋아졌다.
- 하휘 호환성을 제공하는 메서드를 작성 
<br/> → 예전 스타일의 API를 사용중인 코드를 뉴 코드로 쉽게 마이그레이션이 가능하다.

### 기억하자
- 딕셔너리, 긴 튜플, 다른 내장 타입이 복잡하게 내포된 데이터를 값으로 사용하는 딕셔너리를 만들지 말자.
- 완전한 클래스가 제공하는 유연성이 필요하지 않고 가벼운 불변 데이터 컨테이너가 필요한다면 namedtuple을 사용.
- 내부 상태를 표현하는 딕셔너리가 복잡해지면 이 데이터를 관리하는 코드를 여러 클래스로 나눠서 재작성하라.